## 1. Create streamlit app



In [11]:
%%writefile app.py
import streamlit as st
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier, plot_tree
import matplotlib.pyplot as plt

st.set_page_config(page_title="Heart Disease Prediction", layout="wide")
st.title("❤️ Heart Disease Prediction App")

URL = "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data"
COLUMNS = ['age','sex','cp','trestbps','chol','fbs','restecg',
           'thalach','exang','oldpeak','slope','ca','thal','target']
NUMERIC = ['age','trestbps','chol','thalach','oldpeak']
CATEGORICAL = ['sex','cp','fbs','restecg','exang','slope','ca','thal']

@st.cache_resource
def find_best_model():
    df = pd.read_csv(URL, header=None, names=COLUMNS, na_values='?')
    df = df.dropna()
    df['target'] = (df['target'] > 0).astype(int)

    X = df[NUMERIC + CATEGORICAL]
    y = df['target']

    num_pipe = Pipeline([("imputer", SimpleImputer(strategy="median"))])
    cat_pipe = Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
    ])

    preprocessor = ColumnTransformer([
        ("num", num_pipe, NUMERIC),
        ("cat", cat_pipe, CATEGORICAL)
    ])

    pipeline = Pipeline([
        ("preprocessor", preprocessor),
        ("classifier", DecisionTreeClassifier(random_state=42))
    ])

    ### YOUR CODE HERE ###
    param_grid = {
        'classifier__max_depth': [3, 5, 7, 10, None],
        'classifier__min_samples_split': [2, 5, 10],
        'classifier__min_samples_leaf': [1, 2, 4]
    }
    
    grid_search = GridSearchCV(
        pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1
    )
    grid_search.fit(X, y)

    ######################
    return grid_search

with st.sidebar.form("input_form"):
    st.header("Patient Parameters")
    age = st.slider('Age', 20, 90, 50)
    trestbps = st.slider('Resting Blood Pressure (mmHg)', 90, 220, 120)
    chol = st.slider('Serum Cholesterol (mg/dl)', 120, 600, 240)
    thalach = st.slider('Max Heart Rate Achieved', 70, 210, 150)
    oldpeak = st.slider('ST Depression', 0.0, 8.0, 1.0, step=0.1)
    ca = st.slider('Major Vessels colored by Fluoroscopy', 0, 3, 0)

    sex = st.selectbox('Gender', [('Male', 1), ('Female', 0)],
                       format_func=lambda x: x[0])[1]
    cp = st.selectbox('Chest Pain Type', [
        ('Typical Angina', 1), ('Atypical Angina', 2),
        ('Non-anginal Pain', 3), ('Asymptomatic', 4)],
        format_func=lambda x: x[0])[1]
    fbs = st.selectbox('Fasting Blood Sugar > 120 mg/dl',
                       [('False', 0), ('True', 1)],
                       format_func=lambda x: x[0])[1]
    restecg = st.selectbox('Resting ECG', [
        ('Normal', 0), ('ST-T Abnormality', 1), ('LV Hypertrophy', 2)],
        format_func=lambda x: x[0])[1]

    exang = st.selectbox('Exercise-induced Angina', [('No', 0), ('Yes', 1)],
                         format_func=lambda x: x[0])[1]

    slope = st.selectbox('Slope of Peak ST', [
        ('Upsloping', 1), ('Flat', 2), ('Downsloping', 3)],
        format_func=lambda x: x[0])[1]
    thal = st.selectbox('Thalassemia', [
        ('Normal', 3), ('Fixed Defect', 6), ('Reversible Defect', 7)],
        format_func=lambda x: x[0])[1]
    submitted = st.form_submit_button("Predict")

model = find_best_model()

st.write("Enter patient data. The model will predict the likelihood of heart disease.")
st.sidebar.info(f"Optimal model depth found: {model.best_params_['classifier__max_depth']}")

if submitted:
    input_data = pd.DataFrame([{
        'age': age, 'sex': sex, 'cp': cp, 'trestbps': trestbps, 'chol': chol, 'fbs': fbs,
        'restecg': restecg, 'thalach': thalach, 'exang': exang, 'oldpeak': oldpeak,
        'slope': slope, 'ca': ca, 'thal': thal
    }])
    ### YOUR CODE HERE ###
    prediction = model.predict(input_data)
    prob_disease = model.predict_proba(input_data)[0][1]

    ######################

    st.subheader("Prediction Result")
    if prob_disease > 0.5:
        st.error(f"**Disease Likely** — Probability: {prob_disease:.2%}")
    else:
        st.success(f"**Disease Unlikely** — Probability of No Disease: {(1-prob_disease):.2%}")

st.subheader("Model Decision Tree Visualization")

### YOUR CODE HERE ###
classifier = model.best_estimator_.named_steps['classifier']
feature_names = model.best_estimator_.named_steps['preprocessor'].get_feature_names_out()

######################
fig, ax = plt.subplots(figsize=(25, 12))
plot_tree(classifier, feature_names=feature_names,
          class_names=['No Disease', 'Disease'],
          filled=True, rounded=True, ax=ax, fontsize=10)
st.pyplot(fig)


Overwriting app.py


In [12]:
!pip -q install streamlit
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
!chmod +x cloudflared

'wget' is not recognized as an internal or external command,
operable program or batch file.
'chmod' is not recognized as an internal or external command,
operable program or batch file.


In [13]:
!streamlit run app.py \
  --server.port 8501 \
  --server.address 0.0.0.0 \
  --server.headless true \
  --server.enableCORS false \
  --server.enableXsrfProtection false &>/content/logs.txt &

OSError: Background processes not supported.

In [ ]:
!./cloudflared tunnel --url http://localhost:8501 --no-autoupdate

2025-09-19T05:23:29Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-09-19T05:23:29Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-09-19T05:23:34Z INF +--------------------------------------------------------------------------------------------+
2025-09-19T05:23:34Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-09-19T05:23:34Z INF |  https://wines-waterproof-tampa-martin.trycloudflare.c